In [1]:
# 依赖：
# !pip install sentencepiece
# !pip install jieba
# !pip install regex
# !pip install tensorflow
# !pip install tensorflow-hub

In [2]:
import tensorflow_hub as hub
import tensorflow as tf

from tokenization_jieba import JIEBATokenizer

In [3]:
print(tf.__version__)

2.6.0


In [4]:
print(hub.__version__)

0.12.0


In [5]:
tokenizer = JIEBATokenizer(
    'PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.vocab',
    'PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.model')

In [6]:
gpt = hub.load('./pangu-13B-tf2/')

In [7]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_k=50, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数
    length: 输出最大长度
    top_k: 只选择前k个，如果为0则为无限制
    top_p: token的概率排在这以上才有效
    temperature: 温度，如果为1.0则相当于无效
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_k=tf.constant(top_k, tf.int32),
        top_p=tf.constant(top_p, tf.float32),
        temperature=tf.constant(temperature, tf.float32)
    )['output_0']
    return [
        tokenizer.decode([int(x) for x in s]).replace(' ', '')
        for s in ret.numpy()
    ]

# 问答例子

In [8]:
def ask_gpt(question):
    q = f'''
问题：中国首都是哪里？
答案：北京
问题：美国的首都是哪里？
答案：华盛顿
问题：李白在哪个朝代？
答案：唐朝
问题：{question}
答案：'''
    ret = sample(tokenizer, gpt, q, 3, 10, top_k=50, top_p=0.9, temperature=0.9)
    answers = []
    for x in ret:
        a = x[len(q):]
        a = a.split('\n')[0]
        answers.append(a)
    return answers

In [9]:
ask_gpt('唐伯虎是哪个朝代的人？')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.563 seconds.
Prefix dict has been built successfully.


['明代', '明朝', '明朝']

In [10]:
ask_gpt('世界上最帅的是谁？')

['布拉德皮特', '贝克汉姆', '美国总统里根']

In [11]:
ask_gpt('世界上最聪明的人是谁？')

['爱因斯坦', '爱因斯坦', '哥伦布']

In [12]:
ask_gpt('李清照是哪个朝代的人？')

['宋朝', '北宋', '宋朝']

In [13]:
ask_gpt('关公和秦琼谁厉害？')

['关羽', '关羽', '秦琼']

In [14]:
ask_gpt('如何赚一百万美元？')

['把一百元的东西放到地上,然后去掉', '去赌场', '每天去一家公司工作八小时,每小时赚']

In [15]:
ask_gpt('烧烤和火锅哪个好吃？')

['烧烤', '烧烤', '烧烤']

In [16]:
ask_gpt('如何成为老师？')

['拜师学艺', '考试', '先当学生,再当老师。']

In [17]:
ask_gpt('佩奇是什么动物？')

['袋鼠', '熊猫', '猩猩']

In [18]:
ask_gpt('大象有几条腿？')

['八条', '四条腿', '四条腿']

In [19]:
ask_gpt('姚明有多高？')

['姚明1米88', '2米26', '2米26']

In [20]:
ask_gpt('汤姆和杰瑞谁厉害？')

['杰瑞', '杰瑞', '杰瑞']

In [21]:
ask_gpt('野原新之助能打过樱桃子吗？')

['能', '当然。', '能']

In [22]:
ask_gpt('学音乐和学画画哪个好？')

['学音乐', '学音乐好', '音乐']

In [23]:
ask_gpt('中文和英文哪个难学？')

['中文', '中文', '中文']

In [24]:
ask_gpt('python和java哪个难？')

['python', 'java<eot>', 'java']

In [25]:
ask_gpt('“锄禾日当午”的下一句是什么？')

['趁着这棵树刚刚发芽。', '谁知盘中餐,粒粒皆辛苦', '谁知盘中餐,粒粒皆辛苦']

In [26]:
ask_gpt('烤鸭的主要原料是什么？')

['鲜鸭肉', '鸭毛', '鸭肉']

In [27]:
ask_gpt('把大象放冰箱总共分几步？')

['把大象放冰箱之前要做什么?', '两步', '一步<eot>问题:最长的一条被子有多']

In [28]:
ask_gpt('珠穆朗玛峰有多高？')

['8848米', '8844.43米', '8848米']

In [29]:
ask_gpt('世界上最高的山是什么？')

['珠穆朗玛峰', '珠穆朗玛峰', '珠穆朗玛峰']

In [30]:
ask_gpt('中国有多少人？')

['3亿', '中国人口总数为13亿', '3亿人']

In [31]:
ask_gpt('日本有多少人？')

['一亿以上', '33.3', '大约20人']

In [32]:
ask_gpt('苹果手机好还是华为手机好？')

['苹果', '华为手机好', '华为手机好']

# 推理

In [33]:
ret = sample(tokenizer, gpt, '小明决定去吃饭，小红继续写作业\n问题：去吃饭的人是谁？\n答案：', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小明。
小明说:“你自己去
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小明<eot>为什么中国男人这么优秀
这话
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小明<eot>为什么我的电脑老是死机,重装
--------------------


# 英文问答例子

In [34]:
ret = sample(tokenizer, gpt, '默写英文：\n狗dog\n猫cat\n鸟', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写英文:
狗dog
猫cat
鸟duck
鱼noodle
马
--------------------
默写英文:
狗dog
猫cat
鸟birds
牛bull
马
--------------------
默写英文:
狗dog
猫cat
鸟chicken
猴puppy

--------------------


# 默写古诗例子

In [35]:
ret = sample(tokenizer, gpt, '默写古诗：\n白日依山尽，黄河入海流。\n床前明月光，', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
2、写出你
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望明
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望明
--------------------


# 不同的角色对话生成例子

In [36]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n佟掌柜：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
佟掌柜:“小兔崽子!你眼里还有没有我这个老佟!”
李大
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“我的个乖乖,您今儿是怎么了?”
李大嘴:“我
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“李大嘴,你还真叫人难找啊!”
李大嘴:“
--------------------


In [37]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n白展堂：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
白展堂:“李大嘴!”
李大嘴:“展堂!”
白
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“我看你最好还是和小李一起去找他妈妈!”
李大
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“哎哟,您别走啊,您别走啊!”
冯西
--------------------


In [38]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n莫小贝：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
莫小贝:“李大嘴你别这样!”
李大嘴:“莫小贝,你别
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“各回各家,各找各妈!”
李大嘴:“有我呢
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“大家听我说,我们现在是一条船上的人,不能再分开了。大家
--------------------


In [39]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n李白：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
李白:“你可真坏!”
李大嘴:“就我坏,就我坏
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“我有!”
李大嘴:“我有!”
李白
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“我看你是没脸见人了!”
李大嘴:“我这
--------------------


# 问答的例子

In [40]:
ret = sample(tokenizer, gpt, '中国的首都是北京\n日本的首都是东京\n美国的首都是', 3, 3, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
中国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
英国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
法国
--------------------


In [41]:
ret = sample(tokenizer, gpt, '李白所在朝代是唐\n李清照所在朝代是宋\n唐伯虎所在朝代是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明代
--------------------


In [42]:
ret = sample(tokenizer, gpt, '陈奕迅是歌星\n郭德纲是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------


In [43]:
ret = sample(tokenizer, gpt, '珠穆朗玛峰的高度（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

珠穆朗玛峰的高度(米)是4,880,0(5,337
--------------------
珠穆朗玛峰的高度(米)是7,287<eot>珠穆朗玛峰高度
--------------------
珠穆朗玛峰的高度(米)是6,746,4,430<eot>珠
--------------------


In [44]:
ret = sample(tokenizer, gpt, '姚明的身高（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

姚明的身高(米)是多少
姚明身高是2.26米<eot>和
--------------------
姚明的身高(米)是2.01M<eot>姚明的身高(米)是
--------------------
姚明的身高(米)是多少
姚明身高是2.26米<eot>姚明
--------------------


# 算数例子

In [45]:
ret = sample(tokenizer, gpt, '1+1=2\n2+2=4\n3+3=6\n4+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
2+2=4
3+3=6
4+4=9
--------------------
1+1=2
2+2=4
3+3=6
4+4=9
--------------------
1+1=2
2+2=4
3+3=6
4+4=8
--------------------


In [46]:
ret = sample(tokenizer, gpt, '1+1=2\n1+2=3\n1+3=4\n1+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
1+2=3
1+3=4
1+4=6
--------------------
1+1=2
1+2=3
1+3=4
1+4=6
--------------------
1+1=2
1+2=3
1+3=4
1+4=8
--------------------


# ???的例子

In [47]:
ret = sample(tokenizer, gpt, '''惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
''', 3, 30, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
黑光这
黑光这黑光这黑光这黑光这黑光这黑光这黑光这黑光这黑光
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
雷罚天渊之雷
天灵地怒焚天
天劫雷霆怒
雷之极威
雷之极罚
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
这就是境界
我说的是境界啊,惊雷是境界<eot>有多忙
有一个人
人
--------------------


# 写作文例子

In [48]:
ret = sample(tokenizer, gpt, '''爱情是''', 3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱情是什么?
爱是一种感觉,是无法说出的感觉,爱是愉快,是难过,是陶醉,是情绪,是勇敢,是信赖,是诚意,是体贴,
--------------------
爱情是糖,甜到忧伤
这一刻我感到忧伤,原来爱情会给我糖吃<eot>爱情是糖,甜到忧伤,爱情是糖,甜到忧伤!如果说爱情是一种信仰,那么爱情信仰
--------------------
爱情是个什么玩意儿
爱情是你想有就有,想爱就爱,想去就去<eot>爱情是你想有就有,想爱就爱,想去就去爱情是你想有就有,想爱
--------------------


In [49]:
ret = sample(
    tokenizer, gpt,
    '''一时黛玉进了荣府，下了车。众嬷嬷引着，便往东转弯，穿过一个东西的穿堂，向南大厅之后，仪门内大院落，上面五间大正房，两边厢房鹿顶耳房钻山，四通八达，轩昂壮丽，比贾母处不同。黛玉便知这方是正经正内室，一条大甬路，直接出大门的。''',
    3, 200, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。因命紫鹃取钥匙开了仪门,从甬路穿过正院,便进了北向的三间小偏房。两边房门上各有金字横匾,是“蕙兰香芷”四字。那东边一间,也是紫鹃开的,上边⁇了一个“寿”字,两旁配着对联,却是"金满池塘"。黛玉便知这方是专给宝玉安身的,又是个小书房,因指给紫鹃看,说:“你看,他是宝玉的书斋。”那紫鹃见黛玉言语温存,十分欢喜,便忙沏了茶来。<eot>本公司提供北京房产按揭贷款服务,北京房产按揭贷款申请条件,北京房产按揭贷款流程,北京房产按揭贷款服务,北京房产按揭贷款资料,北京房产按揭贷款
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。只听得里面脚步声响,又见两个丫头出来,俱穿着金钏银镯,一个打扮的如牡丹花下芍药,一个打扮的如芙蓉花上芙蓉,走将出来,笑道:“这是谁?好标致!”黛玉忙上前见礼,口内低言:“这是我那大妹子,昨儿晚上,在我这里吃酒,被风吹倒了,也不知道是怎么回事。”那两个小丫头便笑道:“太太说,大姐姐是二姐姐的亲妹子,这两位姐姐才是亲姊妹呢。”黛玉听了,虽也是个大家小姐,毕竟年小,心里颇不自在,见那两个小丫头和两个小丫头站在两边,便向黛玉耳边低低说了几句话,黛玉笑道:“我竟不知你们。”那两个小丫头道:“那是二
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。到了二门口,早有妥当的太监,引着黛玉进了里面的大门,从仪门至东一顺台阶。这是正房,两扇朱漆大门,上有金字匾额,写着“怡红院”三个大字。黛玉正想着,忽见一个太监,引着两个人,从仪门进来,这两个人,一个是宝钗,一个是黛玉。宝玉一见,便笑道:“原来是你两个,来这里做什么?”宝钗笑道:“我

# 对话例子

In [50]:
ret = sample(
    tokenizer, gpt,
    '''A：“今天我想吃火锅”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“今天我想吃火锅”
B:“我想吃披萨”
A:“我想吃辣炒年糕”
B:“我想吃咖喱牛肉”
A:“我想吃龙虾”
B:“我想吃披萨”
--------------------
A:“今天我想吃火锅”
B:“今天我想吃火锅”
C:“今天我想吃火锅”
D:“今天我想吃火锅”
2.下列句子中,没有语病的一句是()
A:从前
--------------------
A:“今天我想吃火锅”
B:“今天我想吃烤鱼”
C:“今天我想吃汉堡”
D:“今天我想吃披萨”
E:“今天我想吃臭豆腐”
F:“今天我想吃
--------------------


In [51]:
ret = sample(
    tokenizer, gpt, '''A：“跟我一起去看电影吧”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“跟我一起去看电影吧”
B:“好啊,但是我要换一部”
A:“好吧,那换一部吧”
B:“好的,太好了”
A:“喂,你的耳朵啊,还没好啊?”
--------------------
A:“跟我一起去看电影吧”
B:“想看什么电影?”
A:“想看《我爱我家》,你陪我一起去吧。”
B:“哦,好吧。”
A:“《我爱我家》是
--------------------
A:“跟我一起去看电影吧”
B:“你喜欢看恐怖片吗?”
A:“你是不是要说我是个变态啊”
B:“是的,我觉得你很变态,你可以去看一下,保证你会喜欢的。”
--------------------


# 对联例子

In [52]:
duilian = '''
上联天，下联地
上联雨，下联风
上联大陆，下联长空
上联雷隐隐，下联雾蒙蒙
上联开心大吉，下联万事亨通
'''

In [53]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联李白作诗，下联',
    5, 2, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联刘禹锡
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联刘禹锡
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联王维写书
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫吟诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联苏轼定
--------------------


In [54]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联追求真爱，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联寻找真缘

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联寻找真爱

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联求平安
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联追求幸福
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联期待良缘

--------------------


In [55]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天天向上，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联日日好运
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联天天快乐
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联年年进步

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联月月平平

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联夜夜失眠
上
--------------------


In [56]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天地在我心，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联日月在我梦
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联自然在我门
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联太阳照乾坤

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联太阳在我手
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联世界在我心
--------------------


# 名人名言

In [57]:
ret = sample(
    tokenizer, gpt, '''鲁迅曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

鲁迅曾经说过:“天才就是百分之九十九的汗水加百分之一的灵感”。我想大多数人都没有注意到,这“百分之一”就是他们对待工作的态度。虽然对待工作的态度不同,但是我们作为领导者,对待
--------------------
鲁迅曾经说过:“在我的周围,是有一种精神的存在。”一个人有什么样的精神,就会有什么样的思想,就会有什么样的行动。一个人拥有怎样的精神,就会形成怎样的性格,就会在日常
--------------------
鲁迅曾经说过:“中国需要有自己的英雄,有自己的‘精神’。”这句话说出了中国人的价值取向。
2018年3月,习近平总书记在“不忘初心、牢记使命”主题教育工作会议上指出
--------------------


In [58]:
ret = sample(
    tokenizer, gpt, '''爱因斯坦曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱因斯坦曾经说过:“如果你想找到正确的科学方法,你必须有一个信仰”。没有信仰,你就不可能有正确的方法。作为一名教师,我更愿意告诉家长和学生,科学方法是每个人都应该懂得的,
--------------------
爱因斯坦曾经说过:“如果一个人能够发现他自己,那么这就是一种解放。”如果一个人能够发现自己,就能使他有可能从经验中不断获得教训,在更高的水平上进行思考。爱因斯坦对此有一句名言
--------------------
爱因斯坦曾经说过:“我不相信命运,命运在我手中。”
命运让你成熟,命运让你坚强,命运让你幸福,
命运让你成熟,命运让你坚强,命运让你幸福,
没有谁能左右
--------------------


In [59]:
ret = sample(
    tokenizer, gpt, '''牛顿曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

牛顿曾经说过:“万有引力是最神奇的力,是一个不可捉摸的力,就像电一样。”科学不相信神,但是科学相信你。科学没有什么定律,但科学相信,你就是你,你可以
--------------------
牛顿曾经说过:“只要你有信心,什么困难都能克服。”
“只要你有信心,什么困难都能克服。”在这世界上,每个人都有自己的目标,有的人的目标就是出人头
--------------------
牛顿曾经说过:“人是理性的生物,是情感的动物。”人类的感情就是情感,一个人生活在社会中,就必然受到社会的影响。而人的这种感情的产生和发展的必然途径是:在生活的过程中
--------------------


In [60]:
ret = sample(
    tokenizer, gpt, '''佛祖曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

佛祖曾经说过:“不为自己求安乐,但愿众生得离苦。”如果我们把“希望众生得到离苦得乐”当作我们人生的目标,就会心甘情愿、乐而不疲、苦而不厌。反之,如果我们为
--------------------
佛祖曾经说过:“一粒沙里见世界,一朵花里见天堂。”其实人生也是这样,面对困难,我们总能迎难而上,但在困难面前,也需要坚强,有一种精神叫⁇撼大树,有一种
--------------------
佛祖曾经说过:“一念为善,一念为恶,直至成佛,为六道众生。”
一念之间,善恶有报。一念之差,天堂地狱。
佛教认为,世界上有三种人,即
--------------------


In [61]:
ret = sample(
    tokenizer, gpt, '''乔布斯曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

乔布斯曾经说过:“如果你还想重新开始,就做一些新的尝试,然后再重复你已经做过的事情。”我想说,我们每个人都应该去尝试一些新的东西,比如在谷歌上面看一看自己喜欢的电影,
--------------------
乔布斯曾经说过:“当我最自豪的时候,我不是在做产品,而是在做我想做的事。”乔布斯还说过:“不管我现在做什么,我总是希望将来能做得更好。”在乔布斯
--------------------
乔布斯曾经说过:“当我进入苹果时,我发现他们(手机)并不是我想要的。我认为手机应该是一个具有特殊性能的产品,应该是我们生活中的一部分。”这句话对产品设计带来了很大的影响,也
--------------------


# 菜谱

In [62]:
ret = sample(
    tokenizer, gpt, '''青椒肉丝的做法：''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

青椒肉丝的做法:1.将猪肉切成丝,青椒切成细丝,用盐和胡椒粉拌匀,腌制10分钟2.锅内放油,烧至七成热时,将肉丝放入锅中滑炒至变色,盛出
--------------------
青椒肉丝的做法:1.猪肉切片用料酒,生抽,老抽,淀粉,盐,蛋清抓匀腌制一会2.青椒洗净去籽,切成细丝3.炒锅烧热,下油
--------------------
青椒肉丝的做法:1.肉丝用盐、酱油、糖、鸡精、淀粉、料酒腌制。青椒洗净去籽,切成细丝。2.锅内倒油,油热后下肉丝翻炒至变色盛出。
--------------------


In [63]:
ret = sample(
    tokenizer, gpt, '''圣女果炒茶叶蛋的做法：''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

圣女果炒茶叶蛋的做法:1.鸡蛋打散,加入适量盐和生粉。<eot>圣女果炒茶叶蛋的做法2.倒入平底锅,炒至鸡蛋变软。<eot>圣女果炒茶叶蛋的做
--------------------
圣女果炒茶叶蛋的做法:是鸡蛋炒制<eot>圣女果炒茶叶蛋的做法:<eot>圣女果炒茶叶蛋的做法:圣女果炒茶叶蛋的做法:圣女果炒茶叶蛋的做法:圣女果炒茶叶
--------------------
圣女果炒茶叶蛋的做法:是圣女果洗净,切成小块<eot>圣女果炒茶叶蛋的做法:<eot>圣女果炒茶叶蛋的做法:主料:圣女果10个、茶叶蛋10个配料:葱段、蒜末
--------------------


In [64]:
ret = sample(
    tokenizer, gpt, '''中国四大发明是''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国四大发明是哪四大发明
造纸术、印刷术、火药、指南针<eot>四大发明分别是指什么呢?一、造纸术造纸术是中国古代四大发明之一,是中国古代劳动人民在总结前人经验的基础上,
--------------------
中国四大发明是指哪些
指南针、造纸术、印刷术、火药<eot>四大发明是指哪些中国四大发明是指中国古代伟大的四大发明。<eot>我的问题是我在网上查的是属于什么版本的游戏呢,
--------------------
中国四大发明是什么
造纸术、印刷术、指南针、火药<eot>2013年,中国有四大发明入选《世界记忆名录》,分别是:造纸术、印刷术、指南针、火药<eot>中国四大发明是什么?<eot>2017
--------------------


In [65]:
ret = sample(
    tokenizer, gpt, '''上联：瑞风播福泽，事业昌盛千家乐
下联：''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

上联:瑞风播福泽,事业昌盛千家乐
下联:福泽传忠信,幸福安康四海人<eot>有朋友来家里做客,他怎么说?
上联:瑞风播福泽,事业昌盛千家乐下联:福泽传忠信,幸福安康四海人
--------------------
上联:瑞风播福泽,事业昌盛千家乐
下联:福瑞迎春,吉星高照万户欢
横批:春满人间
福从天降,福从善来,好运是我们永远的追求。让我们从现在开始,用自己的双手去创造美好
--------------------
上联:瑞风播福泽,事业昌盛千家乐
下联:云帆载福临,顺风顺水百业兴
横批:福满四海<eot>如何做到快速地记住单词?
怎么记住单词的读音?
怎么记住单词的意思?
怎么
--------------------
